In [ ]:
# The plots of vertices and normals in world and camera coordiantes can be found in visualize_mesh.ipynb

###
### Imports
###
from __future__ import print_function, division
import itertools
import os
import torch
import pandas as pd
from skimage import io, transform # package 'scikit-image'
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Import functions from data_loading
import data_loading

# Imports for plotting
import matplotlib.pyplot as plt # Do not use when running on the server
from mpl_toolkits.mplot3d import axes3d # Do not use when running on the server

# Allow the interactive rotation of 3D scatter plots in jupyter notebook
import sys    
import os    
file_name =  os.path.basename(sys.argv[0])
#print(file_name == 'ipykernel_launcher.py') # This basicaly asks whether this file is a jupyter notebook?
if __name__ == "__main__":
    if file_name == 'ipykernel_launcher.py': # Run only in .ipynb, not in exported .py scripts
        get_ipython().run_line_magic('matplotlib', 'notebook') # Equivalent to ''%matplotlib notebook', but it is also understood by .py scripts


In [ ]:
# Instanciating with transform:
if __name__ == '__main__':
    transformed_dataset = data_loading.vertices_Dataset(camera_coordinates = 0,
                                                        transform = 
                                                        transforms.Compose([
                                                            transforms.RandomResizedCrop(224),
                                                            transforms.RandomHorizontalFlip(),
                                                            transforms.ToTensor(),
                                                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                                        ])
                                                       )

    sample = transformed_dataset[22]
    print(type(sample['image']))
    print(sample['image'].shape)

In [ ]:
print(sample['Vertex_coordinates'].shape)

In [ ]:
RT_matrix = np.genfromtxt('RendersTowelWall2/camera_params.txt', delimiter=' ', skip_header=11)
# print('RT matrix:\n', RT_matrix)
# print()

# RT matrix for homogeneous coordinates
zeros_and_1 = np.zeros(4)
zeros_and_1[-1] = 1
zeros_and_1 = np.reshape(zeros_and_1, (1,4))
RT_extended = np.concatenate((RT_matrix, zeros_and_1), axis=0)
# print("RT extended with zeros and 1 below (for homogeneous coordinates):\n", RT_extended)
# print()

(X_camera, Y_camera, Z_camera) = data_loading.world_to_camera_coordinates(sample['Vertex_coordinates'][:,0],
                                                                          sample['Vertex_coordinates'][:,1],
                                                                          sample['Vertex_coordinates'][:,2],
                                                                          RT_extended)

In [ ]:
X_camera_col = np.reshape(X_camera, (X_camera.size, 1))
Y_camera_col = np.reshape(Y_camera, (Y_camera.size, 1))
Z_camera_col = np.reshape(Z_camera, (Z_camera.size, 1))
print(X_camera_col.shape)

In [ ]:
sample['Vertex_coordinates'] = np.hstack((X_camera_col, Y_camera_col, Z_camera_col))
print(sample['Vertex_coordinates'].shape)

In [ ]:
(nX_camera, nY_camera, nZ_camera) = data_loading.world_to_camera_coordinates_normals(sample['normal_coordinates'][:,0],
                                                                          sample['normal_coordinates'][:,1],
                                                                          sample['normal_coordinates'][:,2],
                                                                          RT_matrix)
nX_camera_col = np.reshape(nX_camera, (nX_camera.size, 1))
nY_camera_col = np.reshape(nY_camera, (nY_camera.size, 1))
nZ_camera_col = np.reshape(nZ_camera, (nZ_camera.size, 1))
print(nX_camera_col.shape)

sample['normal_coordinates'] = np.hstack((nX_camera_col, nY_camera_col, nZ_camera_col))
print(sample['normal_coordinates'].shape)